This Jupyter Notebook only demonstrates how to parse AICC situation reports (2025 version) using both Pandas and Polars, and how to download them from the AICC site. For calculating tallies, see `firetally.ipynb`.

In [ ]:
import pandas as pd
import polars as pl
import datetime as dt
from pathlib import Path
import requests

In [ ]:
DATADIR = Path().absolute().parents[1]  / 'data/AICC_reports'
testfn = 'AK_SituationReportExport_20250702.xlsx'
DATADIR

PosixPath('/Users/chris/Projects/FiWePs/data/AICC_reports')

In [ ]:
pl.read_excel(DATADIR / testfn)

OBJECTID,Fire Number,Unique Fire ID,Incident Name,Acres,Fire Cause,Incident Type,Management Option,Protecting Org,Protecting Office,Owner ID,Ownership at Origin,Discovery Date,Containment Date,Control Date,Out Date,Fire Code,DOF Code,USFS Code,Complex,Last updated,Status,Personnel,Narrative Date,Narrative,Latitude,Longitude
i64,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,f64,f64
1,"""372""","""2025-AKGAD-000372""","""Lynx""",0.1,"""Undetermined""","""WF-Wildfire""","""Modified (Jul 10)""","""AFS""","""Galena Zone (GAD)""","""ANCSA""","""AKNAN-NANA Regional Corporatio…","""7/2/2025 21:01""",null,null,null,"""S469""",null,"""PDS469""",null,"""7/2/2025 23:17""","""Unstaffed/Uncontained""",0,"""7/2/2025""","""No narrative.""",67.013,-156.67305
2,"""370""","""2025-AKFAS-511370""","""Healy Road Coal Seam""",3.0,"""Natural""","""WF-Wildfire""","""Full""","""DOF""","""Fairbanks Area (FAS)""","""State""","""AKDNS-AK DNR-Mental Health Tru…","""7/2/2025 16:08""",null,null,null,"""S457""","""7351137025""","""PNS457""",null,"""7/2/2025 16:26""","""Unstaffed/Uncontained""",0,"""7/2/2025""","""While conducting a reconnaissa…",63.861167,-148.761833
3,"""371""","""2025-AKKKS-503371""","""Sandra""",0.01,"""Human""","""WF-Wildfire""","""Critical""","""DOF""","""Kenai-Kodiak Area (KKS)""","""Private""","""AKPRI-Private""","""7/2/2025 15:04""","""7/2/2025 15:53""","""7/2/2025 15:53""",null,"""S45S""","""7350337125""","""PNS45S""",null,"""7/2/2025 19:06""","""Staffed/Contained""",4,"""7/2/2025""","""On 7/2/25 @ 1502 KESA dispatch…",59.742817,-151.266017
4,"""368""","""2025-AKFAS-511368""","""Farmers Market""",0.1,"""Natural""","""WF-Wildfire""","""Critical""","""DOF""","""Fairbanks Area (FAS)""","""Private""","""AKPRI-Private""","""7/2/2025 12:00""","""7/2/2025 17:28""","""7/2/2025 17:29""",null,"""S44B""","""7351136825""","""PNS44B""",null,"""7/2/2025 16:36""","""Unstaffed/Contained""",0,"""7/2/2025""","""At approximately 1200, FECC re…",64.863134,-147.782293
5,"""369""","""2025-AKTAS-513369""","""F/A Big Creek""",0.0,"""Undetermined""","""FA-False Alarm""","""Limited""","""DOF""","""Tok Area (TAS)""","""State""","""AKDNS-AK Dept. Natural Resourc…","""7/2/2025 11:38""",null,null,null,null,"""7351336925""",null,null,null,null,null,null,null,63.5299,-141.4319
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
404,null,"""2025-AKMSS-250001""","""CY25 Sunset RX""",0.01,"""Human""","""RX-Prescribed Fire""","""Critical""","""DOF""","""Mat-Su Area (MSS)""","""State""","""AKDNS-AK Dept. Natural Resourc…","""1/2/2025 12:36""",null,null,null,null,null,null,null,"""7/2/2025 06:41""",null,0,"""7/2/2025""","""No narrative.""",61.660133,-149.602283
405,null,"""2025-AKFAS-250712""","""2025 Nenana Ridge RX""",0.1,"""Human""","""RX-Prescribed Fire""","""Full""","""DOF""","""Fairbanks Area (FAS)""","""State""","""AKDNS-AK Dept. Natural Resourc…","""1/1/2025 08:47""",null,null,null,null,null,null,null,"""7/2/2025 06:28""",null,0,"""7/2/2025""","""No narrative.""",64.653617,-148.69
406,null,"""2025-AKKKS-250003""","""CY25 Wharf Ave RX""",0.01,"""Human""","""RX-Prescribed Fire""","""Critical""","""DOF""","""Kenai-Kodiak Area (KKS)""","""State""","""AKDNS-AK DNR-Mental Health Tru…","""1/1/2025 08:00""",null,null,null,null,null,null,null,"""7/2/2025 06:41""",null,0,"""7/2/2025""","""No narrative.""",60.487802,-150.866415


In [ ]:
pd.read_excel(DATADIR / testfn)

,OBJECTID,Fire Number,Unique Fire ID,Incident Name,Acres,Fire Cause,Incident Type,Management Option,Protecting Org,Protecting Office,...,DOF Code,USFS Code,Complex,Last updated,Status,Personnel,Narrative Date,Narrative,Latitude,Longitude
0,1,372.0,2025-AKGAD-000372,Lynx,0.10,Undetermined,WF-Wildfire,Modified (Jul 10),AFS,Galena Zone (GAD),...,NaN,PDS469,NaN,7/2/2025 23:17,Unstaffed/Uncontained,0.0,7/2/2025,No narrative.,67.013000,-156.673050
1,2,370.0,2025-AKFAS-511370,Healy Road Coal Seam,3.00,Natural,WF-Wildfire,Full,DOF,Fairbanks Area (FAS),...,7.351137e+09,PNS457,NaN,7/2/2025 16:26,Unstaffed/Uncontained,0.0,7/2/2025,"While conducting a reconnaissance mission, N64...",63.861167,-148.761833
2,3,371.0,2025-AKKKS-503371,Sandra,0.01,Human,WF-Wildfire,Critical,DOF,Kenai-Kodiak Area (KKS),...,7.350337e+09,PNS45S,NaN,7/2/2025 19:06,Staffed/Contained,4.0,7/2/2025,On 7/2/25 @ 1502 KESA dispatched resources to ...,59.742817,-151.266017
3,4,368.0,2025-AKFAS-511368,Farmers Market,0.10,Natural,WF-Wildfire,Critical,DOF,Fairbanks Area (FAS),...,7.351137e+09,PNS44B,NaN,7/2/2025 16:36,Unstaffed/Contained,0.0,7/2/2025,"At approximately 1200, FECC reported an active...",64.863134,-147.782293
4,5,369.0,2025-AKTAS-513369,F/A Big Creek,0.00,Undetermined,FA-False Alarm,Limited,DOF,Tok Area (TAS),...,7.351337e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.529900,-141.431900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,404,NaN,2025-AKMSS-250001,CY25 Sunset RX,0.01,Human,RX-Prescribed Fire,Critical,DOF,Mat-Su Area (MSS),...,NaN,NaN,NaN,7/2/2025 06:41,NaN,0.0,7/2/2025,No narrative.,61.660133,-149.602283
404,405,NaN,2025-AKFAS-250712,2025 Nenana Ridge RX,0.10,Human,RX-Prescribed Fire,Full,DOF,Fairbanks Area (FAS),...,NaN,NaN,NaN,7/2/2025 06:28,NaN,0.0,7/2/2025,No narrative.,64.653617,-148.690000
405,406,NaN,2025-AKKKS-250003,CY25 Wharf Ave RX,0.01,Human,RX-Prescribed Fire,Critical,DOF,Kenai-Kodiak Area (KKS),...,NaN,NaN,NaN,7/2/2025 06:41,NaN,0.0,7/2/2025,No narrative.,60.487802,-150.866415
406,407,NaN,2025-AKKKS-250001,CY25 Morgan''s Landing RX,8.00,Human,RX-Prescribed Fire,Critical,DOF,Kenai-Kodiak Area (KKS),...,7.323600e+09,NaN,NaN,7/2/2025 06:38,NaN,0.0,7/2/2025,No narrative.,60.503372,-150.863667


In [ ]:
URLtemplate_A = "https://fire.ak.blm.gov/content/aicc/Previous%20Situation%20Reports/2025%20Situtation%20Report%20Exports/"
URLtemplate_C = "AK_SituationReportExport_"

# https://fire.ak.blm.gov/content/aicc/Previous%20Situation%20Reports/2025%20Situtation%20Report%20Exports/04_2025/AK_SituationReportExport_20250423.xlsx
# https://fire.ak.blm.gov/content/aicc/Previous%20Situation%20Reports/2025%20Situtation%20Report%20Exports/2025_04/AK_SituationReportExport_20250419.xlsx


fntemplate = 'AK_SituationReportExport_'

In [ ]:
for item in pd.date_range('20250415',dt.datetime.now().strftime("%Y%m%d"), freq='d'):
    thedate = item.strftime('%Y%m%d')
    URL = URL = f"{URLtemplate_A}{item.strftime('%m_%Y')}/{URLtemplate_C}{thedate}.xlsx"
    response = requests.get(URL)
    if response.status_code == 200:
        print(URL)
        with open(DATADIR / f"{fntemplate}{thedate}.xlsx", 'wb') as dst:
            dst.write(response.content)
    else:
        URL = f"{URLtemplate_A}{URLtemplate_C}{thedate}.xlsx"
        print(URL)
        response = requests.get(URL)
        if response.status_code == 200:
            with open(DATADIR / f"{fntemplate}{thedate}.xlsx", 'wb') as dst:
                dst.write(response.content)


https://fire.ak.blm.gov/content/aicc/Previous%20Situation%20Reports/2025%20Situtation%20Report%20Exports/04_2025/AK_SituationReportExport_20250415.xlsx
https://fire.ak.blm.gov/content/aicc/Previous%20Situation%20Reports/2025%20Situtation%20Report%20Exports/04_2025/AK_SituationReportExport_20250416.xlsx
https://fire.ak.blm.gov/content/aicc/Previous%20Situation%20Reports/2025%20Situtation%20Report%20Exports/04_2025/AK_SituationReportExport_20250417.xlsx
https://fire.ak.blm.gov/content/aicc/Previous%20Situation%20Reports/2025%20Situtation%20Report%20Exports/04_2025/AK_SituationReportExport_20250418.xlsx
https://fire.ak.blm.gov/content/aicc/Previous%20Situation%20Reports/2025%20Situtation%20Report%20Exports/04_2025/AK_SituationReportExport_20250419.xlsx
https://fire.ak.blm.gov/content/aicc/Previous%20Situation%20Reports/2025%20Situtation%20Report%20Exports/04_2025/AK_SituationReportExport_20250420.xlsx
https://fire.ak.blm.gov/content/aicc/Previous%20Situation%20Reports/2025%20Situtation%20